<a href="https://colab.research.google.com/github/Disco-Gnome/CSC84030_Fall23/blob/main/7_mining_data_stream_(bloom_filter)_RGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In this Colab we just need to install a [bloom_filter](https://github.com/hiway/python-bloom-filter), a Python library which offers an implementations of Bloom filters.  Run the cell below!

In [2]:
!pip install bloom_filter

### Data Loading

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the very first spell-checking programs in Unix-like operating systems.

In [3]:
import nltk
nltk.download('words')

from nltk.corpus import words
word_list = words.words()
print(f'Dictionary length: {len(word_list)}')
print(word_list[:15])

[nltk_data] Downloading package words to /root/nltk_data...


Dictionary length: 236736
['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron', 'Aaronic', 'Aaronical', 'Aaronite', 'Aaronitic', 'Aaru']


[nltk_data]   Unzipping corpora/words.zip.


Then we load another dataset from the NLTK Corpora collection: ```movie_reviews```.

The movie reviews are categorized between *positive* and *negative*, so we construct a list of words (usually called **bag of words**) for each category.

In [4]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
  neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
  pos_reviews.extend(movie_reviews.words(fileid))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


### Your task

In this Colab, you will develop a very simplistic spell-checker.  By no means you should think of using it for a real-world use case, but it is an interesting exercise to highlight the strenghts and weaknesses of Bloom Filters!

In [5]:
from bloom_filter import BloomFilter

word_filter = BloomFilter(max_elements=236736)

for word in word_list:
  word_filter.add(word)

word_set = set(word_list)

If you executed the cell above, you now have 3 different variables in your scope:

1.   ```word_list```, a Python list containing the English dictionary (in case insensitive order)
2.   ```word_filter```, a Bloom filter where we have already added all the words in the English dictionary
3.   ```word_set```, a [Python set](https://docs.python.org/3.6/library/stdtypes.html#set-types-set-frozenset) built from the same list of words in the English dictionary

Let's inspect the size of each data structure using the [getsizeof()](https://docs.python.org/3/library/sys.html#sys.getsizeof) method!



In [21]:
#Inspecting byte size of alternative structures
from sys import getsizeof

print(f'Size of word_list (in bytes): {getsizeof(word_list)}')
print(f'Size of word_filter (in bytes): {getsizeof(word_filter)}')
print(f'Size of word_set (in bytes): {getsizeof(word_set)}')
print('\nFrom largest to smallest: set, list, filter')
print('\nA Bloom Filter takes up a mere 48 bytes: 2.34e-3% as much memory as a list, and 5.72e-4% as much memory as a set.')

Size of word_list (in bytes): 2055512
Size of word_filter (in bytes): 48
Size of word_set (in bytes): 8388824

From largest to smallest: set, list, filter

A Bloom Filter takes up a mere 48 bytes: 2.34e-3% as much memory as a list, and 5.72e-4% as much memory as a set.


You should have noticed how efficient is the Bloom filter in terms of memory footprint!

Now let's find out how fast is the main operation for which we construct Bloom filters: *membership testing*. To do so, we will use the ```%timeit``` IPython magic command, which times the repeated execution of a single Python statement.

In [15]:
#Timing queries on various structures

%timeit -r 3 "California" in word_list
%timeit -r 3 "California" in word_filter
%timeit -r 3 "California" in word_set

print('\nFrom fastest to slowest: set, filter, list.')
print('\nQuerying a Bloom Filter is roughly 3,264% times faster than querying a list, but ~22,790% slower than querying a set.')
print('\nFrom these last 2 cells, we observe that a Bloom Filter splits the difference, being faster to query than a list,\nthough slower to query than a set. But, it also uses much less memory to store than either a list or set.')

413 µs ± 3.98 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)
16.4 µs ± 4.89 µs per loop (mean ± std. dev. of 3 runs, 100000 loops each)
76.3 ns ± 23.7 ns per loop (mean ± std. dev. of 3 runs, 10000000 loops each)

From fastest to slowest: set, filter, list.

Querying a Bloom Filter is roughly 3,264% times faster than querying a list, but ~22,790% slower than querying a set.

From these last 2 cells, we observe that a Bloom Filter splits the difference, being faster to query than a list,
though slower to query than a set. But, it also uses much less memory to store than either a list or set.


Notice the performance gap between linear search on a list, multiple hash computations in a Bloom filter, and a single hash computation in a native Python ```Set()```.

We now have all the building blocks required to build our spell-checker, and we understand the performance tradeoffs of each data structure we chose. Write a function that takes as arguments (1) a list of words, and (2) any of the 3 dictionary data structures we constructed. The function must return the number of words which **do not appear** in the dictionary.

In [26]:
#Define our function
def countNotInDict(words_object, dict_struct):
    assert type(words_object) == list
    count_not_found = 0
    for word in words_object:
        if word not in dict_struct:
            count_not_found = count_not_found + 1
    return count_not_found
    del count_not_found

In [54]:
#Test our function
test_list = ['A', 'aardvark', 'Aaru', 'asdf1234', 'jkl;5678']
assert countNotInDict(test_list, word_filter) == 2
print(f'Test words not found in dictionary: {countNotInDict(test_list, word_filter)}\n')

%timeit -r 3 countNotInDict(test_list, word_list)
%timeit -r 3 countNotInDict(test_list, word_filter)
%timeit -r 3 countNotInDict(test_list, word_set)

print('\nFrom fastest to slowest: set, filter, list. Same as above.')

Test words not found in dictionary: 2

10.9 ms ± 2.42 ms per loop (mean ± std. dev. of 3 runs, 100 loops each)
44 µs ± 637 ns per loop (mean ± std. dev. of 3 runs, 10000 loops each)
474 ns ± 6.13 ns per loop (mean ± std. dev. of 3 runs, 1000000 loops each)

From fastest to slowest: set, filter, list. Same as above.


In [53]:
#Run our function
print('_____Running Function on negative Reviews Words List_____')
print(f'Words in negative reviews: {len(neg_reviews)}')
print(f'Words in negative reviews not found in dictionary: {(countNotInDict(neg_reviews, word_set))}')
print(f'Proportion of words in negative reviews not found in dictionary: {round(100*countNotInDict(neg_reviews, word_set)/len(neg_reviews),2)}%')
print('\n_____Running Function on Positive Reviews Words List_____')
print(f'Words in positive reviews: {len(pos_reviews)}')
print(f'Words in positive reviews not found in dictionary: {(countNotInDict(pos_reviews, word_set))}')
print(f'Proportion of words in positive reviews not found in dictionary: {round(100*countNotInDict(pos_reviews, word_set)/len(neg_reviews),2)}%')

_____Testing Function on Negative Reviews Words List_____
Words in negative reviews: 751256
Words in negative reviews not found in dictionary: 210258
Proportion of words in negative reviews not found in dictionary: 27.99%

_____Testing Function on Positive Reviews Words List_____
Words in positive reviews: 832564
Words in positive reviews not found in dictionary: 232436
Proportion of words in positive reviews not found in dictionary: 30.94%


#### **Submission Intruction:**

#### Click File -> Download -> Download **.ipynb**, and upload the downloaded file to Blackboard.